In [ ]:
import numpy as np
import pandas as pd
import json
from path import Path
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
import matplotlib.pyplot as plt

In [ ]:
# set variable equal to loaded dataset
tracks_df = pd.read_csv('popular_songs.csv', error_bad_lines=False)

#tracks_df = pd.read_csv(file)
#tracks_df.shape

In [ ]:
tracks_df.head()

In [ ]:
tracks_df.describe().transpose()

In [ ]:
#get rid of white space before and after each column name
tracks_df.columns = tracks_df.columns.str.strip()
tracks_df.head()

In [ ]:
new_tracks_df = tracks_df.drop(['song','artist','release_date','genres','popular_date'], axis =1)

# Drop the null columns where all values are null
new_tracks_df = new_tracks_df.dropna(axis='columns', how='all')

# Drop the null rows
new_tracks_df = new_tracks_df.dropna()

new_tracks_df.tail()

In [ ]:
wob_df = pd.DataFrame(new_tracks_df['weeks_on_board'])
wob_df.describe()

In [ ]:
# Create a box-and-whisker plot for the number of weeks on the board.
# Outliers are values that are above 36 weeks on the board.

x_labels = ["Weeks On Board"]
fig, ax = plt.subplots(figsize=(20,26))
ax.boxplot(wob_df, labels=x_labels)
# Add the title, y-axis label and grid
ax.set_title("Top 100 Billboard")
ax.set_ylabel("Number of weeks")
ax.set_yticks(np.arange(0, 100, step=1.0))
ax.grid()
plt.show()

In [ ]:
header_list = ['popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'ranking', 'weeks_on_board', 'target']
new_tracks_df = new_tracks_df.reindex(columns = header_list)
new_tracks_df.head()

In [ ]:
# The average number of weeks that a song is on the billboard is 12 weeks.
# Create a for loop that determines if the row value in weeks_on_board column is greater than or equal to 12 and assign
# it a new value of 0 and add it to the 'target' column and everything else change to a 1.
for index in new_tracks_df.index:
    if new_tracks_df.loc[index,'weeks_on_board']>=12:
        new_tracks_df.loc[index,'target'] = 0
    else:
        new_tracks_df.loc[index,'target'] = 1

In [113]:
new_tracks_df.head()

,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,ranking,weeks_on_board,target
0,76.0,0.0906,0.499,0.800,0.000000,0.1470,-2.665,0.0502,139.919,70,19,0.0
1,78.0,0.0334,0.721,0.716,0.000840,0.2370,-7.037,0.0657,104.994,67,8,1.0
2,71.0,0.1880,0.846,0.748,0.000000,0.0930,-3.512,0.0780,90.006,68,6,1.0
3,45.0,0.2950,0.803,0.715,0.000134,0.0574,-3.280,0.2980,101.085,69,7,1.0
4,71.0,0.0360,0.445,0.492,0.000000,0.1220,-5.717,0.0294,148.013,75,3,1.0


In [115]:
target_tracks_df = new_tracks_df.drop(columns = "weeks_on_board")
target_tracks_df.head()

,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,ranking,target
0,76.0,0.0906,0.499,0.800,0.000000,0.1470,-2.665,0.0502,139.919,70,0.0
1,78.0,0.0334,0.721,0.716,0.000840,0.2370,-7.037,0.0657,104.994,67,1.0
2,71.0,0.1880,0.846,0.748,0.000000,0.0930,-3.512,0.0780,90.006,68,1.0
3,45.0,0.2950,0.803,0.715,0.000134,0.0574,-3.280,0.2980,101.085,69,1.0
4,71.0,0.0360,0.445,0.492,0.000000,0.1220,-5.717,0.0294,148.013,75,1.0


# Use the Nueral Network MLPClassifier ML Model for the target_tracks_df
    1. This dataframe excludes the number of weeks on the board and instead uses a binary outcome. This binary values was determined by if the weeks on board is greater than or equal to 12 then change to 0, else change to 1.
    

### Create features and split into training and testing 

In [116]:
# Create our features
X = target_tracks_df.drop(columns = "target")
#X = pd.get_dummies(X)

# Create our target
y = target_tracks_df["target"]

In [117]:
X.describe()

,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,ranking
count,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000
mean,63.208398,0.166375,0.646721,0.676498,0.010053,0.178256,-5.936540,0.101920,122.542894,50.471037
std,16.218658,0.209721,0.135772,0.166208,0.076090,0.135081,2.303629,0.102682,28.346675,28.863363
min,0.000000,0.000014,0.076800,0.002310,0.000000,0.020000,-42.887000,0.022500,48.718000,1.000000
25%,57.000000,0.021900,0.557000,0.567000,0.000000,0.094500,-7.031000,0.037800,99.643000,25.000000
50%,66.000000,0.075400,0.652000,0.698000,0.000000,0.124000,-5.603000,0.055400,122.504000,50.000000
75%,73.000000,0.228000,0.742000,0.805000,0.000011,0.224000,-4.421000,0.121000,140.309000,75.000000
max,97.000000,0.996000,0.981000,0.996000,0.973000,0.989000,0.175000,0.951000,208.067000,100.000000


In [118]:
# Check the balance of our target values
y.value_counts()

1.0    29045
0.0    23350
Name: target, dtype: int64

In [119]:
# train the data
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state = 1)

In [120]:
# def accuracy(confusion_matrix):
#     diagonal_sum = confusion_matrix.trace()
#     sum_of_all_elements = confusion_matrix.sum()
#     return diagonal_sum / sum_of_all_elements

X_train.shape

(39296, 10)

In [121]:
X_test.shape

(13099, 10)

### Standardization: StandardScaler

In [122]:
scaler = StandardScaler()

In [123]:
train_scaled = scaler.fit_transform(X_train)

In [124]:
test_scaled = scaler.transform(X_test)

In [125]:
#Initializing the MLPClassifier

model = MLPClassifier(solver='lbfgs', random_state=42)

In [126]:
#Fitting the training data to the network
model.fit(train_scaled, y_train)

C:\Users\kbead\anaconda3\envs\mlenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(random_state=42, solver='lbfgs')

In [127]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(test_scaled)

In [128]:
accuracy_score(y_train, model.predict(train_scaled))*100

72.12947882736157

In [129]:
accuracy_score(y_test, y_pred)*100

70.28017405908848

In [130]:
cm = confusion_matrix
cm(y_test, y_pred)

array([[3801, 2077],
       [1816, 5405]], dtype=int64)

# Use the Nueral Network MLPClassifier ML Model for the new_tracks_df
    1. This dataframe includes the number of weeks on the board and uses the binary values that were determined by if the weeks on board is greater than or equal to 12 then change to 0, else change to 1.
    

### Create features and split into training and testing 

In [131]:
# Create our features
X = new_tracks_df.drop(columns = "target")
#X = pd.get_dummies(X)

# Create our target
y = new_tracks_df["target"]

In [132]:
X.describe()

,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,ranking,weeks_on_board
count,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000,52395.000000
mean,63.208398,0.166375,0.646721,0.676498,0.010053,0.178256,-5.936540,0.101920,122.542894,50.471037,12.278118
std,16.218658,0.209721,0.135772,0.166208,0.076090,0.135081,2.303629,0.102682,28.346675,28.863363,10.130632
min,0.000000,0.000014,0.076800,0.002310,0.000000,0.020000,-42.887000,0.022500,48.718000,1.000000,1.000000
25%,57.000000,0.021900,0.557000,0.567000,0.000000,0.094500,-7.031000,0.037800,99.643000,25.000000,4.000000
50%,66.000000,0.075400,0.652000,0.698000,0.000000,0.124000,-5.603000,0.055400,122.504000,50.000000,10.000000
75%,73.000000,0.228000,0.742000,0.805000,0.000011,0.224000,-4.421000,0.121000,140.309000,75.000000,17.000000
max,97.000000,0.996000,0.981000,0.996000,0.973000,0.989000,0.175000,0.951000,208.067000,100.000000,87.000000


In [133]:
# Check the balance of our target values
y.value_counts()

1.0    29045
0.0    23350
Name: target, dtype: int64

In [134]:
# train the data
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state = 1)

In [135]:
X_train.shape

(39296, 11)

In [136]:
X_test.shape

(13099, 11)

In [137]:
scaler = StandardScaler()

In [138]:
train_scaled = scaler.fit_transform(X_train)

In [139]:
test_scaled = scaler.transform(X_test)

In [140]:
#Initializing the MLPClassifier

model = MLPClassifier(solver='lbfgs', random_state=42)

In [141]:
#Fitting the training data to the network
model.fit(train_scaled, y_train)

MLPClassifier(random_state=42, solver='lbfgs')

In [142]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(test_scaled)

In [143]:
accuracy_score(y_train, model.predict(train_scaled))*100

100.0

In [144]:
accuracy_score(y_test, y_pred)*100

100.0

In [145]:
cm = confusion_matrix
cm(y_test, y_pred)

array([[5878,    0],
       [   0, 7221]], dtype=int64)